In [1]:
# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
from pandas import DataFrame
import numpy as np
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [8]:
df_hotel = pd.read_csv("finanzas-hotel-bookings.csv", index_col=0)
df_hotel.head()

/home/miriam/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,5,13,14,15,16,20,21,26,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,False,342.0,2015.0,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,Resort Hotel,False,737.0,NaN,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
2,Resort Hotel,False,7.0,2015.0,July,27.0,1.0,0.0,1.0,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
3,Resort Hotel,False,13.0,NaN,July,27.0,1.0,0.0,1.0,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
4,Resort Hotel,False,14.0,NaN,July,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN


In [9]:
df_hotel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182877 entries, 0 to 182876
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  object 
 2   lead_time                       119490 non-null  float64
 3   arrival_date_year               64829 non-null   float64
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        101004 non-null  float64
 6   arrival_date_day_of_month       119271 non-null  float64
 7   stays_in_weekend_nights         119490 non-null  float64
 8   stays_in_week_nights            119490 non-null  float64
 9   adults                          119428 non-null  float64
 10  children                        69302 non-null   float64
 11  babies                          119431 non-null  float64
 12  meal            

In [11]:
info_columnas = pd.concat([df_hotel.isnull().sum(), (df_hotel.isnull().sum()/df_hotel.shape[0]) * 100, df_hotel.dtypes], axis=1)
info_columnas.columns = ['Total de Nulos','Porcentaje de Nulos', 'Tipos de Columnas']
info_columnas

,Total de Nulos,Porcentaje de Nulos,Tipos de Columnas
hotel,63487,34.715683,object
is_canceled,63487,34.715683,object
lead_time,63387,34.661002,float64
arrival_date_year,118048,64.550490,float64
arrival_date_month,63487,34.715683,object
arrival_date_week_number,81873,44.769435,float64
arrival_date_day_of_month,63606,34.780754,float64
stays_in_weekend_nights,63387,34.661002,float64
stays_in_week_nights,63387,34.661002,float64
adults,63449,34.694904,float64


In [12]:
df_hotel["hotel"].unique()

array(['Resort Hotel', 'City Hotel', nan], dtype=object)

In [13]:
df_hotel["is_canceled"].value_counts()

False    75166
True     44224
Name: is_canceled, dtype: int64

In [14]:
df_hotel.duplicated().sum()

63040

In [15]:
def exploracion_csv(archivo):
  
    #Echamos un vistazo
    print("Primeras filas:")
    display(df_hotel.head(10))
    print("-----------------------------")

    print("Últimas filas:")
    display(df_hotel.tail(10))
    print("-----------------------------")

    print("Filas aleatorias:")
    display(df_hotel.sample(10))
    print("-----------------------------")

    print("El df tiene {df_hotel.shape[0]} filas y {df_hotel.shape[1]} columnas")
    print("-----------------------------")

    print("Tipos de datos y nulos:")
    display(df_hotel.info())
    print("-----------------------------")
    
    print(f"Características columnas númericas:\n{df_hotel.describe().T}")
    print(f"-----------------------------")

    print(f"Características columnas de texto:\n{df_hotel.describe (include='object').T}")
    print(f"-----------------------------")

    print(f"Duplicados:\n{df_hotel.duplicated().sum()}")
    print(f"-----------------------------")

    # generamos un DataFrame para los valores nulos
    print("Los nulos que tenemos en el conjunto de datos son:")
    df_nulos = pd.DataFrame(df_hotel.isnull().sum() / df_hotel.shape[0] * 100, columns = ["%_nulos"])
    display(df_nulos[df_nulos["%_nulos"] > 0])

    print("\n ..................... \n")
    print("Los valores que tenemos para las columnas categóricas son: ")
    df_categoricas = df_hotel.select_dtypes(include = "O")
    
    for col in df_categoricas.columns:
        print(f"La columna {col.upper()} tiene las siguientes valores únicos:")
        display(pd.DataFrame(df_hotel[col].value_counts()/df_hotel[col].shape[0])*100)  

In [16]:
exploracion_csv("finanzas-hotel-bookings.csv")

Primeras filas:


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,False,342.0,2015.0,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,Resort Hotel,False,737.0,NaN,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
2,Resort Hotel,False,7.0,2015.0,July,27.0,1.0,0.0,1.0,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
3,Resort Hotel,False,13.0,NaN,July,27.0,1.0,0.0,1.0,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
4,Resort Hotel,False,14.0,NaN,July,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
5,Resort Hotel,False,14.0,NaN,July,27.0,1.0,0.0,2.0,2.0,NaN,0.0,BB,GBR,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,NaN,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
6,Resort Hotel,False,0.0,2015.0,July,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Direct,Direct,0.0,NaN,0.0,C,C,0.0,NaN,NaN,0.0,NaN,107.0,0.0,0.0,Check-Out,2015-07-03 00:00:00,NaN
7,Resort Hotel,False,9.0,2015.0,July,27.0,1.0,0.0,2.0,2.0,0.0,0.0,FB,NaN,Direct,Direct,0.0,NaN,0.0,C,C,0.0,303.0,NaN,0.0,NaN,103.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
8,Resort Hotel,True,85.0,2015.0,July,27.0,1.0,0.0,3.0,2.0,0.0,0.0,BB,PRT,NaN,NaN,0.0,NaN,0.0,A,A,0.0,240.0,NaN,0.0,Transient,82.0,0.0,1.0,Canceled,2015-05-06 00:00:00,NaN
9,Resort Hotel,True,75.0,2015.0,July,27.0,1.0,0.0,3.0,2.0,0.0,0.0,HB,NaN,NaN,TA/TO,0.0,0.0,0.0,D,D,0.0,15.0,NaN,0.0,Transient,105.5,0.0,0.0,Canceled,2015-04-22 00:00:00,NaN


-----------------------------
Últimas filas:


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
182867,NaN,NaN,390.304129,NaN,NaN,NaN,NaN,11.458344,23.291221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182868,NaN,NaN,222.703566,NaN,NaN,NaN,NaN,11.193862,18.600639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182869,NaN,NaN,325.951764,NaN,NaN,NaN,NaN,15.295969,24.246056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182870,NaN,NaN,295.615115,NaN,NaN,NaN,NaN,18.333015,20.181327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182871,NaN,NaN,231.480575,NaN,NaN,NaN,NaN,18.880637,16.523465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182872,NaN,NaN,224.337762,NaN,NaN,NaN,NaN,19.005545,19.819823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182873,NaN,NaN,390.141963,NaN,NaN,NaN,NaN,14.751794,19.989726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182874,NaN,NaN,230.689826,NaN,NaN,NaN,NaN,11.409496,20.461372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182875,NaN,NaN,304.888534,NaN,NaN,NaN,NaN,16.744472,15.400773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182876,NaN,NaN,341.238166,NaN,NaN,NaN,NaN,17.791486,24.442690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


-----------------------------
Filas aleatorias:


...

,reservation_status
Check-Out,41.101943
Canceled,23.522367
No-Show,0.660006


La columna RESERVATION_STATUS_DATE tiene las siguientes valore únicos:


,reservation_status_date
2015-10-21 00:00:00,0.709767
2015-07-06 00:00:00,0.400269
2016-11-25 00:00:00,0.378396
2015-01-01 00:00:00,0.370741
2016-01-18 00:00:00,0.307310
...,...
2029-04-31,0.000547
2022-09-31,0.000547
2029-02-30,0.000547
2029-06-31,0.000547


La columna 0 tiene las siguientes valore únicos:


,0
0.0,6.988304
2.0,1.292125
1.0,1.159796
A,0.968410
BB,0.687347
...,...
48.2,0.000547
149.5,0.000547
68.07,0.000547
153.07,0.000547
